In [1]:
import rospy
import numpy as np
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from smach import State, StateMachine
import smach_ros
from tf.transformations import euler_from_quaternion
from time import sleep

In [13]:
pos = Twist()
break_v = 0
turn_b = 0
pub = rospy.Publisher('/cmd_vel',Twist,queue_size=10)
rospy.init_node('fsm_ra')
rate = rospy.Rate(10)
puntos = [[0,1],[1,1],[1,0],[0,0]]

In [14]:
puntos

[[0, 1], [1, 1], [1, 0], [0, 0]]

In [4]:
def callbackOdom(msg):
    global pos
    pos.linear.x = msg.pose.pose.position.x
    pos.linear.y = msg.pose.pose.position.y
    rot_q = msg.pose.pose.orientation
    (_,_,pos.angular.z) = euler_from_quaternion([rot_q.x, rot_q.y, rot_q.z, rot_q.w])

In [5]:
def smallest_angle_diff(t,s):
    a = t - s
    a -= 2*np.pi if a > np.pi else -2*np.pi if a < -np.pi else 0
    return a

In [6]:
def callbackScan(msg):
    global break_v
    global turn_b
    scan = np.array(msg.ranges)
    ang = msg.angle_min - np.pi
    andf = msg.angle_max - np.pi
    inc = msg.angle_increment
    angles = np.arange(ang,andf+inc,inc)
    angles = angles[np.r_[-60:60]]
    scan_f = scan[np.r_[-60:60]]
    angles = angles[~np.isnan(scan_f)]
    scan_f = scan_f[~np.isnan(scan_f)]
    angles = angles[np.isfinite(scan_f)]
    scan_f = scan_f[np.isfinite(scan_f)]
    # Máximo alcanze 0.12 - recomendada 0.15
    if scan_f[scan_f < 0.35].shape[0] > 0:
        break_v = (scan_f.min() - 0.23)/scan_f.min()
        try:
            angle = smallest_angle_diff(angles[scan_f == scan_f.min()],np.pi)
            turn_b = angle + (4*np.pi)/9 if angle < 0 else angle - (4*np.pi)/9
        except:
            turn_b = 0
    else:
        break_v = 1
        turn_b = 0

In [7]:
class Turn(State):
    def __init__(self):
        State.__init__(self, outcomes=['turnr','fowardr','success'])
    def execute(self, ud):
        global pos
        global pub
        global rate
        global puntos
        if len(puntos) == 0:
            return 'success'
        goal = puntos[0]
        print(goal)
        vel = Twist()
        ang = np.arctan2(goal[1]-pos.linear.y,goal[0]-pos.linear.x)
        vel.angular.z = smallest_angle_diff(ang,pos.angular.z)*2
        vel.angular.z = vel.angular.z if abs(vel.angular.z) <= 2.84 else 2.84*np.sign(vel.angular.z)
        pub.publish(vel)
        rate.sleep()
        if abs(smallest_angle_diff(ang,pos.angular.z)) > 0.01:
            return 'turnr'
        else:
            vel.angular.z = 0
            pub.publish(vel)
            rate.sleep()
            pub.publish(vel)
            rate.sleep()
            return 'fowardr'

In [8]:
class Foward(State):
    def __init__(self):
        State.__init__(self, outcomes=['turnr','fowardr','success'])
    def execute(self, ud):
        global pos
        global pub
        global rate
        global puntos
        global break_v
        global turn_b
        goal = puntos[0]
        vel = Twist()
        ang = np.arctan2(goal[1]-pos.linear.y,goal[0]-pos.linear.x)
        vel.angular.z = smallest_angle_diff(ang,pos.angular.z)*2 if turn_b == 0 else turn_b*(1-break_v) if break_v >= 0 else 0
        vel.angular.z = vel.angular.z if abs(vel.angular.z) <= 2.84 else 2.84*np.sign(vel.angular.z)
        dis = np.sqrt((pos.linear.x - goal[0])**2 + (pos.linear.y - goal[1])**2)
        vel.linear.x = dis*(break_v)
        vel.linear.x = vel.linear.x if abs(vel.linear.x) <= 0.22 else 0.22*np.sign(vel.linear.x)*abs(break_v)
        pub.publish(vel)
        rate.sleep()
        if dis > 0.01:
            return 'fowardr'
        else:
            puntos.pop(0)
            vel.angular.z = 0
            vel.linear.x = 0
            pub.publish(vel)
            rate.sleep()
            pub.publish(vel)
            rate.sleep()
            if len(puntos) == 0:
                return 'success'
            else:
                return 'turnr'

In [9]:
def main():
    global pos
    global rate
    
    odom = rospy.Subscriber('/odom',Odometry,callbackOdom)
    scanS = rospy.Subscriber('/scan',LaserScan,callbackScan)
    
    sm = StateMachine(outcomes=['succeeded'])
    sm.userdata.sm_input = 0

    # Open the state machine container. A state machine container holds a number of states.
    with sm:
        
        # Add states to the container, and specify the transitions between states
        # For example, if the outcome of state LOCKED is 'coin', then we transition to state UNLOCKED.
        StateMachine.add('TURN', Turn(), transitions={'turnr':'TURN','fowardr':'FOWARD','success':'succeeded'})
        StateMachine.add('FOWARD', Foward(), transitions={'turnr':'TURN','fowardr':'FOWARD','success':'succeeded'})
    
    # View our state transitions using ROS by creating and starting the instrospection server
    sis = smach_ros.IntrospectionServer('server_name', sm, '/SM_ROOT')
    sis.start()
    
    # Execute the state machine 
    outcome = sm.execute()
    print(outcome)
    # Wait for ctrl-c to stop the application
    if outcome == 'success':
        sis.stop()
    else:
        sleep(1)

In [15]:
if __name__ == '__main__':
    try:
        main()
    except rospy.ROSInterruptException:
        pass

[INFO] [1681617241.485837, 965.099000]: State machine starting in initial state 'TURN' with userdata: 
	['sm_input']
[0, 1]
[INFO] [1681617241.487425, 965.102000]: State machine transitioning 'TURN':'turnr'-->'TURN'
[0, 1]
[INFO] [1681617241.596128, 965.202000]: State machine transitioning 'TURN':'turnr'-->'TURN'
[0, 1]
[INFO] [1681617241.688711, 965.302000]: State machine transitioning 'TURN':'turnr'-->'TURN'
[0, 1]
[INFO] [1681617241.787866, 965.402000]: State machine transitioning 'TURN':'turnr'-->'TURN'
[0, 1]
[INFO] [1681617241.888327, 965.502000]: State machine transitioning 'TURN':'turnr'-->'TURN'
[0, 1]
[INFO] [1681617241.990256, 965.602000]: State machine transitioning 'TURN':'turnr'-->'TURN'
[0, 1]
[INFO] [1681617242.089576, 965.702000]: State machine transitioning 'TURN':'turnr'-->'TURN'
[0, 1]
[INFO] [1681617242.189608, 965.802000]: State machine transitioning 'TURN':'turnr'-->'TURN'
[0, 1]
[INFO] [1681617242.289591, 965.902000]: State machine transitioning 'TURN':'turnr'--